# Zillow Preprocessing and Training Data<a id='5_Modeling'></a>


In our prior notebook we cleaned the data and reviewed the relationship between features.
Now, we will focus on identifiend any categorical features to create dummy features, standarize our features, and creating train and test splits. 

Just a reminder that our goal is to predict the Median Listing Price of homes in the state of AZ

In [76]:
#Importing modules needed 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold



###  Loading Data

In [77]:
# Importing data

ZillowData = pd.read_csv('../Capstone_2/Zillow_EDA_df.csv')
ZillowData.head()

,CountyName-StateName-RegionName-Date,DaysOnZillow_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,MedianListingPricePerSqft_AllHomes,MedianListingPricePerSqft_SingleFamilyResidence,...,MedianPctOfPriceReduction_SingleFamilyResidence,PctOfHomesDecreasingInValues_AllHomes,PctOfHomesIncreasingInValues_AllHomes,PctOfHomesSellingForGain_AllHomes,PctOfHomesSellingForLoss_AllHomes,PctOfListingsWithPriceReductions_AllHomes,PctOfListingsWithPriceReductions_SingleFamilyResidence,PriceToRentRatio_AllHomes,Sale_Counts,Year
0,Kent-Delaware-10001-1996-04-30,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,...,4.198062,32.16,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996
1,New Castle-Delaware-10003-1996-04-30,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,...,4.198062,32.16,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996
2,Baldwin-Alabama-1003-1996-04-30,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,...,4.198062,32.16,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996
3,Blount-Alabama-1009-1996-04-30,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,...,4.198062,32.16,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996
4,Chambers-Alabama-1017-1996-04-30,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,...,4.198062,32.16,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996


Just a reminder that our goal is to predict the Median Listing Price of homes in the state of AZ

In [78]:
# Splitting the 'CountyName-StateName-RegionName-Date' to perform One-hot encoding on State Name
#We will extract Date in a seperate column.
ZillowData['Date'] = ZillowData['CountyName-StateName-RegionName-Date'].str[-10:]
ZillowData['CountyName-StateName-RegionName'] = ZillowData['CountyName-StateName-RegionName-Date'].str[:-11]

# Obtaining the State Name
ZillowData = ZillowData.drop(columns=['CountyName-StateName-RegionName-Date'], axis=1)
ZillowData['State'] = ZillowData['CountyName-StateName-RegionName'].str.rsplit('-', 2).str[-2]
ZillowData = ZillowData.drop(columns=['CountyName-StateName-RegionName'], axis=1)

# Getting Month feature from Date

ZillowData['Date'] = pd.to_datetime(ZillowData['Date'])
ZillowData['Month'] = ZillowData['Date'].dt.month
ZillowData = ZillowData.drop(columns=['Date'], axis=1)



In [79]:
ZillowData.head()

,DaysOnZillow_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,MedianListingPricePerSqft_AllHomes,MedianListingPricePerSqft_SingleFamilyResidence,MedianListingPrice_1Bedroom,...,PctOfHomesIncreasingInValues_AllHomes,PctOfHomesSellingForGain_AllHomes,PctOfHomesSellingForLoss_AllHomes,PctOfListingsWithPriceReductions_AllHomes,PctOfListingsWithPriceReductions_SingleFamilyResidence,PriceToRentRatio_AllHomes,Sale_Counts,Year,State,Month
0,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996,Delaware,4
1,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996,Delaware,4
2,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996,Alabama,4
3,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996,Alabama,4
4,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,58.67,92.27,7.73,11.253197,11.386139,9.91,151.0,1996,Alabama,4


## One-hot encoding<a id='4.6_Train/Test_Split'></a>

Before splittin the data inot train/test sets, we need to create new feature for our categorical column 'CountyName-StateName-RegionName'

In [80]:
# Perform one-hot encoding
encoder = OneHotEncoder()
one_hot_encoded = encoder.fit_transform(ZillowData[['State']])

# Convert the encoded data to a DataFrame
one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoder.get_feature_names(['State']))

# Concatenate the one-hot encoded data with the original data
ZillowData = pd.concat([ZillowData, one_hot_encoded_df], axis=1)

# Dropping 'State' column to be able to scale data

ZillowData = ZillowData.drop(columns=['State'], axis=1)


C:\Users\EdwinPalomo\Anaconda_Packages\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [81]:
ZillowData.head()

,DaysOnZillow_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,MedianListingPricePerSqft_AllHomes,MedianListingPricePerSqft_SingleFamilyResidence,MedianListingPrice_1Bedroom,...,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.0,159450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We have succesfully created dummy variables for the State names columns. Now we our dataframe contains 78 columns, but not all of these features are helpful to predict the meding houseing price. In our next step we will work on feature selection before we Train/Split our data.

## Feature Selection<a id='4.6_Train/Test_Split'></a>

Multiple techniques exist for feature selection from a data frame, however, this notebook focuses on variance as the criterion. Specifically, features with low or zero variance will be excluded.

In [82]:
# Creating the VarianceThreshold class with a threshold to 0.01
selector = VarianceThreshold(threshold=0.01)

# Fit the selector to our Zillow data set
selector.fit(ZillowData.drop(columns='MedianListingPrice_AllHomes'))

# Transforming the data to only keep features with variance less than 1%
X_high_variance = selector.transform(ZillowData.drop(columns='MedianListingPrice_AllHomes'))

# review the new number of features on our df
X_high_variance.shape

(518791, 59)

We now need to know the name of the features to be able to drop features with variance below 1%.

In [85]:
# Creating a variable to save feature names
feature_names = ZillowData.drop(columns='MedianListingPrice_AllHomes').columns

# Creating a boolean mask indicating which features have been selected
mask = selector.get_support()

# Create a list of the names of the selected features
selected_feature_names = [feature_names[i] for i in range(ZillowData.drop(columns='MedianListingPrice_AllHomes').shape[1]) if mask[i]]

# Resizing our features df.
X_high_variance = ZillowData[selected_feature_names]
X_high_variance.shape

(518791, 59)

The number of features was reduced from 78 to 59 through the exclusion of features with variance below 1% of the maximum variance.
Now we can move forward to Train/Test split our data.

## Train/Test Split<a id='4.6_Train/Test_Split'></a>

We will start by spliting our data into train and test variable. We will partition the data 70/30

In [86]:
# spliting the data into 70/30 partitions
X_train, X_test, y_train, y_test = train_test_split(X_high_variance, 
                                                    ZillowData.MedianListingPrice_AllHomes, test_size=0.3, 
                                                    random_state=47)

In [87]:
# high overview of partitined data
X_train.shape, X_test.shape

((363153, 59), (155638, 59))

In [88]:
# high overview of partitined data

y_train.shape, y_test.shape

((363153,), (155638,))

In [90]:
# high overview of partitined data

X_train.head()

,DaysOnZillow_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,MedianListingPricePerSqft_AllHomes,MedianListingPricePerSqft_SingleFamilyResidence,MedianListingPrice_1Bedroom,...,State_Oklahoma,State_Oregon,State_Pennsylvania,State_South Carolina,State_Tennessee,State_Texas,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin
491522,94.750,1406.0,126.720192,136.217949,152.423469,172.315978,189.229765,158.536585,159.722222,159450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
124190,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.000000,159450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140512,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.000000,159450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21135,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.000000,159450.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
249143,110.125,319.0,187.849166,114.035088,100.641026,112.187731,125.451825,95.209465,95.000000,159450.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##  Scaling Data<a id='4.7_Initial_Not-Even-A-Model'></a>


Now that we have splitted the data we will standarze the features (zero mean and unit variance). 
This will help our improve the performance of our model 

In [91]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on the training data
scaler.fit(X_train)

# Transform the training and test data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

##  Summary <a id='4.7_Initial_Not-Even-A-Model'></a>


On this step we where able to perfrom the below steps:

1.Create dummy variables for the state names

2.Perform feature selection by removing features with variance below 1%.

3.Scale and split our data into train and test sets 

Now our data is ready for modeling, in our next steps we will review 4 different modeling technices and asses their performance. 

